In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-PtII").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_FR_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_FR = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_multilingual_FR_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df_FR.show()

In [ ]:
from pyspark.sql.functions import col , upper , initcap

In [ ]:
df_FR.printSchema()
df_FR.count()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



254080

In [ ]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 

df_FR_High_Votes=df_FR.filter("total_votes>=20").select(df_FR.columns[:15]) ##.show()

In [ ]:
df_FR_High_Votes.printSchema()

In [ ]:
df_FR_High_Votes.columns
df_FR_High_Votes.describe()
df_FR_High_Votes.count()

11000

filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

In [ ]:
df_FR_High_Votes.withColumn('happy_votes',df_FR_High_Votes['helpful_votes']).show()

In [ ]:
helpful_votes_df=df_FR_High_Votes.withColumn("happy_votes",df_FR_High_Votes["helpful_votes"]/df_FR_High_Votes["total_votes"]*100) ##.show()

In [ ]:
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|       happy_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|         FR|      47266|R2LX6LJ7BA92OO|B00YN6XHMU|     474654167|Grey: Fifty Shade...|Digital_Ebook_Pur...|          1|           21|         27|   N|                Y|De la guimauve en...|Je pensais que la...|2015-06-20 00:00:00| 77.777777777777

In [ ]:
helpful_votes_df.count()

11000

In [ ]:
happy_helpful_votes_df=helpful_votes_df.filter("happy_votes>=.50") #.select(df_FR.columns[:15]) ##.show()

In [ ]:
happy_helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|       happy_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|         FR|      47266|R2LX6LJ7BA92OO|B00YN6XHMU|     474654167|Grey: Fifty Shade...|Digital_Ebook_Pur...|          1|           21|         27|   N|                Y|De la guimauve en...|Je pensais que la...|2015-06-20 00:00:00| 77.777777777777

In [ ]:
happy_helpful_votes_df.count()

10942

create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

In [ ]:
Vine_Yes_df=happy_helpful_votes_df.filter("vine=='Y'")

In [ ]:
Vine_Yes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|       happy_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|         FR|   21965371|R22SUWPP3KRZT6|B00KRM8LE8|     711168977|Amazon - Étui de ...|              PC|          3|           21|         26|   Y|                N|bien mais 50 % de...|Sur la conception...|2014-10-19 00:00:00| 80.76923076923077|
|         FR

In [ ]:
Vine_Yes_df.count()

19

all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

In [ ]:
Vine_No_df=happy_helpful_votes_df.filter("vine=='N'")

In [ ]:
Vine_No_df.show()
Vine_No_df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|       happy_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|         FR|      47266|R2LX6LJ7BA92OO|B00YN6XHMU|     474654167|Grey: Fifty Shade...|Digital_Ebook_Pur...|          1|           21|         27|   N|                Y|De la guimauve en...|Je pensais que la...|2015-06-20 00:00:00| 77.777777777777

10923

total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

#Yes Vine 

In [ ]:
#total number of reviews
df_YReview = Vine_Yes_df.filter(Vine_Yes_df.review_headline.isNotNull())

In [ ]:
df_YReview.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|       happy_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|         FR|   21965371|R22SUWPP3KRZT6|B00KRM8LE8|     711168977|Amazon - Étui de ...|              PC|          3|           21|         26|   Y|                N|bien mais 50 % de...|Sur la conception...|2014-10-19 00:00:00| 80.76923076923077|
|         FR

In [ ]:
Vine_Reviews=df_YReview.count()

In [ ]:
print(Vine_Reviews)

19


In [ ]:
df_YReview.filter(df_YReview.star_rating == '5').show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+-----------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|      happy_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+-----------------+
|         FR|   24680976|R30LQPCSI3ELHN|2226246940|     927755747|            22/11/63|           Books|          5|           18|         23|   Y|                N|Retour vers le passé|Difficle de comme...|2013-02-18 00:00:00|78.26086956521739|
|         FR|   

In [ ]:
vine5star=df_YReview.filter(df_YReview.star_rating == '5').count()
print(vine5star)

9


In [ ]:
percent5star=vine5star/Vine_Reviews*100
print(percent5star)

47.368421052631575


#No Vine

In [ ]:
print(df_NReview.filter(col("review_body").isNull || col("review_body") === "").count())_NReview

In [ ]:
hasReview=df_NReview.filter(col("review_date").isNotNull()).count()
print(hasReview)

10923


In [ ]:
noReview=df_NReview.filter(col("review_date").isNull()).count()
print(noReview)

0


In [ ]:
df_NReview = Vine_No_df.filter(Vine_No_df.review_headline.isNotNull())

In [ ]:
df_NReview.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|       happy_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+------------------+
|         FR|      47266|R2LX6LJ7BA92OO|B00YN6XHMU|     474654167|Grey: Fifty Shade...|Digital_Ebook_Pur...|          1|           21|         27|   N|                Y|De la guimauve en...|Je pensais que la...|2015-06-20 00:00:00| 77.777777777777

In [ ]:
NonVine_Reviews=df_NReview.count()
print(NonVine_Reviews)

10923


In [ ]:
nonvine5star=df_NReview.filter(df_NReview.star_rating == '5').count()
print(nonvine5star)

5194


In [ ]:
percent5starNV=nonvine5star/NonVine_Reviews*100
print(percent5starNV)

47.55103909182459
